### Project Phase 2
In this phase, we:
1. implement ss and sh algorithm
2. check Nash equilibrium

In [28]:
import random
import numpy as np

# tree node
class Node:
    def __init__(self, index):
        self.index = index
        self.color = 0 # color can be 0 or 1, red is 0 and blue is 1
        self.subtree_count = 0
        self.red_subtree_count = 0
        self.blue_subtree_count = 0

class Simulation:
    def __init__(self, random_seed = 14):
        self.nodes = []
        self.tree = {}
        self.F = set()
        self.P = set()
        self.count = 0
        self.random_seed = random_seed

    def __del__(self):
        print("end simulation")

    # construct the tee
    def construct_tree(self, file_name):
        def read_data(file_name):
            with open(file_name) as f:
                lines = f.readlines()
            str_list_lines = [line.rstrip("\n").rstrip(" ").lstrip(" ").split(" ") for line in lines]
            int_list_lines = []
            for str_list in str_list_lines:
                try:
                    int_list_lines.append(list(map(int, str_list)))
                except:
                    raise Exception("Invalid input format!")
            return int_list_lines
        int_list_lines = read_data(file_name)
        num_nodes = int_list_lines[0][0]
        self.count = num_nodes
        # map node with index of node
        for i in range(num_nodes+1):
            self.nodes.append(Node(i))
        # create a tree structure (in which all the node is represented with integer)
        self.tree = dict.fromkeys([i+1 for i in range(num_nodes)])
        for int_list in int_list_lines[1:]:
            if len(int_list) == 1:
                self.tree[int_list[0]] = None # leaf of the tree
            else:
                self.tree[int_list[0]] = int_list[1:]

    # initialize the game by assigning the colors
    def init_game(self):
        current_seed = self.random_seed
        # initialize the random sequence
        for i in range(len(self.nodes)):
            self.nodes[i].color = random.Random(current_seed).randint(0, 1)
            current_seed += 1

    # update the subtree_count of all the nodes in the tree
    def update_subtree_count_tree(self):
        def update_subtree_count(node_index):
            if self.tree[node_index] is None:
                self.nodes[node_index].subtree_count = 0
                return 0, 0, 0
            current_subtree_count = 0
            current_red_subtree_count = 0
            current_blue_subtree_count = 0
            for child_index in self.tree[node_index]:
                current_subtree_count += 1
                if self.nodes[child_index].color == 0: current_red_subtree_count += 1
                else: current_blue_subtree_count += 1
                last_subtree_count, last_red_subtree_count, last_blue_subtree_count = update_subtree_count(child_index)
                current_subtree_count += last_subtree_count
                current_red_subtree_count += last_red_subtree_count
                current_blue_subtree_count += last_blue_subtree_count
            self.nodes[node_index].subtree_count = current_subtree_count
            self.nodes[node_index].red_subtree_count = current_red_subtree_count
            self.nodes[node_index].blue_subtree_count = current_blue_subtree_count
            return current_subtree_count, current_red_subtree_count, current_blue_subtree_count
        update_subtree_count(1)

    # infection
    def infect_round(self, F_last):
        """
        :param F_last: (set(int)) infected node indexes of last iteration
        :return F_new: (set(int)) newly infected node indexes
        """
        F_new = set() # newly infected nodes
        for index in F_last:
            if self.tree[index] is not None:
                for child in self.tree[index]:
                    if child not in self.P:
                        F_new.add(child)
        self.F = self.F.union(F_new)
        return F_new

    # vaccinate
    def vaccinate(self, node_index):
        # we cannot vaccinate node which is already in P
        self.P.add(node_index)

    # degree_heavy_implementation
    def selfish(self, infected_indexes, curren_color):
        """
        :param infected_indexes: (int) the current infected
        :return: vac_index: (int) the index of the node to be vacinated
        """
        max_child_num = 0
        max_child_index = -1
        for node_index in infected_indexes:
            if self.tree[node_index] is None:
                continue
            for neighbor_index in self.tree[node_index]:
                if neighbor_index not in self.P and neighbor_index not in self.F:
                    if self.tree[neighbor_index] is not None:
                        if curren_color == 0:
                            current_child_num = self.nodes[neighbor_index].red_subtree_count - self.nodes[neighbor_index].blue_subtree_count
                        else:
                            current_child_num = self.nodes[neighbor_index].blue_subtree_count - self.nodes[neighbor_index].red_subtree_count
                        if current_child_num > max_child_num:
                            max_child_num = current_child_num
                            max_child_index = neighbor_index
        if max_child_index == -1:
            return None
        self.P.add(max_child_index)
        return max_child_index

    def subtree_heavy(self, infected_indexes):
        """
        :param infected_indexes: (int) the current infected
        :return: vac_index: (int) the index of the node to be vacinated
        """
        max_child_num = 0
        max_child_index = -1
        for node_index in infected_indexes:
            if self.tree[node_index] is None:
                continue
            for neighbor_index in self.tree[node_index]:
                if neighbor_index not in self.P and neighbor_index not in self.F:
                    if self.tree[neighbor_index] is not None:
                        current_child_num = self.nodes[neighbor_index].subtree_count
                        if current_child_num > max_child_num:
                            max_child_num = current_child_num
                            max_child_index = neighbor_index
        if max_child_index == -1:
            return None
        self.P.add(max_child_index)
        return max_child_index

    # simulate: ss and ss
    def ss_ss_simulate(self, input_file):
        self.construct_tree(input_file)
        self.init_game()
        self.update_subtree_count_tree()
        F_new = set()
        F_new.add(1)
        self.F.add(1)
        player = random.Random(self.random_seed).randint(0, 1)
        while len(F_new) > 0:
            vac_index = self.selfish(F_new, player)
            F_new = self.infect_round(F_new)
            player = (player + 1) % 2
        # calculate payoff
        total_set = set(range(1, self.count + 1))
        saved_set = total_set.difference(self.F)
        red_count = 0
        blue_count = 0
        for node_index in saved_set:
            if self.nodes[node_index].color == 0: red_count += 1
            else: blue_count += 1
        payoff_red = red_count - blue_count
        payoff_blue = blue_count - red_count
        # print result
        print("ss-ss strategy:")
        print("red payoff: ",payoff_red)
        print("blue payoff: ", payoff_blue)
        print("infected number: ", len(self.F))
        print("total number: ", self.count)
        print("infected portion: ", len(self.F) / self.count)
        return payoff_red, payoff_blue

    # simulate: ss and sh
    def ss_sh_simulate(self, input_file):
        self.construct_tree(input_file)
        self.init_game()
        self.update_subtree_count_tree()
        F_new = set()
        F_new.add(1)
        self.F.add(1)
        player = random.Random(self.random_seed).randint(0, 1)
        while len(F_new) > 0:
            if player == 0: vac_index = self.selfish(F_new, player)
            else: vac_index = self.subtree_heavy(F_new)
            F_new = self.infect_round(F_new)
            player = (player + 1) % 2
        # calculate payoff
        total_set = set(range(1, self.count + 1))
        saved_set = total_set.difference(self.F)
        red_count = 0
        blue_count = 0
        for node_index in saved_set:
            if self.nodes[node_index].color == 0: red_count += 1
            else: blue_count += 1
        payoff_red = red_count - blue_count
        payoff_blue = blue_count - red_count
        # print result
        print("ss-sh strategy:")
        print("red payoff: ",payoff_red)
        print("blue payoff: ", payoff_blue)
        print("infected number: ", len(self.F))
        print("total number: ", self.count)
        print("infected portion: ", len(self.F) / self.count)
        return payoff_red, payoff_blue

    # simulate: sh and ss
    def sh_ss_simulate(self, input_file):
        self.construct_tree(input_file)
        self.init_game()
        self.update_subtree_count_tree()
        F_new = set()
        F_new.add(1)
        self.F.add(1)
        player = random.Random(self.random_seed).randint(0, 1)
        while len(F_new) > 0:
            if player == 1: vac_index = self.selfish(F_new, player)
            else: vac_index = self.subtree_heavy(F_new)
            F_new = self.infect_round(F_new)
            player = (player + 1) % 2
        # calculate payoff
        total_set = set(range(1, self.count + 1))
        saved_set = total_set.difference(self.F)
        red_count = 0
        blue_count = 0
        for node_index in saved_set:
            if self.nodes[node_index].color == 0: red_count += 1
            else: blue_count += 1
        payoff_red = red_count - blue_count
        payoff_blue = blue_count - red_count
        # print result
        print("sh-ss strategy:")
        print("red payoff: ",payoff_red)
        print("blue payoff: ", payoff_blue)
        print("infected number: ", len(self.F))
        print("total number: ", self.count)
        print("infected portion: ", len(self.F) / self.count)
        return payoff_red, payoff_blue

    # simulate: sh and sh
    def sh_sh_simulate(self, input_file):
        self.construct_tree(input_file)
        self.init_game()
        self.update_subtree_count_tree()
        F_new = set()
        F_new.add(1)
        self.F.add(1)
        player = random.Random(self.random_seed).randint(0, 1)
        while len(F_new) > 0:
            vac_index = self.subtree_heavy(F_new)
            F_new = self.infect_round(F_new)
            player = (player + 1) % 2
        # calculate payoff
        total_set = set(range(1, self.count + 1))
        saved_set = total_set.difference(self.F)
        red_count = 0
        blue_count = 0
        for node_index in saved_set:
            if self.nodes[node_index].color == 0: red_count += 1
            else: blue_count += 1
        payoff_red = red_count - blue_count
        payoff_blue = blue_count - red_count
        # print result
        print("sh-sh strategy:")
        print("red payoff: ",payoff_red)
        print("blue payoff: ", payoff_blue)
        print("infected number: ", len(self.F))
        print("total number: ", self.count)
        print("infected portion: ", len(self.F) / self.count)
        return payoff_red, payoff_blue

def run_game(file_name, seed):
    red_matrix = np.zeros((2,2))
    blue_matrix = np.zeros((2,2))
    simulation_ss_ss = Simulation(seed)
    r_00, b_00 = simulation_ss_ss.ss_ss_simulate(file_name)
    simulation_ss_ss.__del__()
    simulation_ss_sh = Simulation(seed)
    r_01, b_01 = simulation_ss_sh.ss_sh_simulate(file_name)
    simulation_ss_sh.__del__()
    simulation_sh_ss = Simulation(seed)
    r_10, b_10 = simulation_sh_ss.sh_ss_simulate(file_name)
    simulation_sh_ss.__del__()
    simulation_sh_sh = Simulation(seed)
    r_11, b_11 = simulation_sh_sh.sh_sh_simulate(file_name)
    simulation_sh_sh.__del__()
    red_matrix[0,0] = r_00
    red_matrix[0,1] = r_01
    red_matrix[1,0] = r_10
    red_matrix[1,1] = r_11
    blue_matrix[0,0] = b_00
    blue_matrix[0,1] = b_01
    blue_matrix[1,0] = b_10
    blue_matrix[1,1] = b_11
    return red_matrix, blue_matrix

if __name__ == '__main__':
    file_name = "testCase.txt"
    red = np.zeros((2,2))
    blue = np.zeros((2,2))
    seed = 0
    for i in range(1, 11):
        current_file_name = file_name.split(".")[0] + str(i) + ".txt"
        red_matrix, blue_matrix = run_game(current_file_name, seed)
        red = red + red_matrix
        blue = blue + blue_matrix
        seed += 1
    red = red / 10
    blue = blue / 10
    print(red)
    print(blue)

ss-ss strategy:
red payoff:  534
blue payoff:  -534
infected number:  102975
total number:  228627
infected portion:  0.4504061200120721
end simulation
ss-sh strategy:
red payoff:  472
blue payoff:  -472
infected number:  2603
total number:  228627
infected portion:  0.011385356935095155
end simulation
sh-ss strategy:
red payoff:  330
blue payoff:  -330
infected number:  100449
total number:  228627
infected portion:  0.4393575561941503
end simulation
sh-sh strategy:
red payoff:  411
blue payoff:  -411
infected number:  1090
total number:  228627
infected portion:  0.00476759087946743
end simulation
end simulation
end simulation
end simulation
end simulation
ss-ss strategy:
red payoff:  384
blue payoff:  -384
infected number:  80773
total number:  249433
infected portion:  0.3238264383622055
end simulation
ss-sh strategy:
red payoff:  767
blue payoff:  -767
infected number:  48658
total number:  249433
infected portion:  0.19507442880452866
end simulation
sh-ss strategy:
red payoff:  -